# Lesson 14 - Dynamic Polymorphism

This notebook supports the materials in [lesson 14 of C++ for Finance](https://loz-hurst.github.io/cpp-finance-materials-new/lessons/lesson-14.html).

## Forward declarations

It is a good idea to include a reference to where the full definition can be found in the comment, so save you searching for it later.

In [ ]:
// This class is in MyClasses.hpp
class ForwardDeclaredClass;

In [ ]:
void SomeFunction(ForwardDeclaredClass & cls) {
    // Do something that does not "use" ForwardDeclaredClass
}

You cannot do anything that requires knowing the insides of the class if you only have a forward declaration.  This will cause a compile-time error:

In [ ]:
void SomeOtherFunction(ForwardDeclaredClass & cls) {
    cls.SomeFunction();
}

Be careful of subtle cases where you are using (as opposed to referencing) a class, and therefore cannot use a forward declaration.  This is also a compile-time error because passing by value involves creating (using a constructor) a new instance:

In [ ]:
void SomeOtherFunction(ForwardDeclaredClass cls) {
    // Do something that does not "use" ForwardDeclaredClass
}

Even if your object uses the default constructor, C++ needs the definition to know for sure that another constructor has not been provided.

## Polymorphism

### Static and dynamic type, and virtual functions refresher

First include the iostream header, so we can see what is happening:

In [ ]:
#include <iostream>

Here is an example of a small class hierachy, with a virtual and non-virtual function in the base class that is overridden in two derrived classes:

In [ ]:
// A class
class MyClass {
private:
    int i_;
public:
    MyClass() : i_{0} {};
    // A pure virtual function
    virtual void VirtualFunction() const = 0;
    // A function that is not virtual
    void NotVirtualFunction() const {std::cout << "MyClass::NotVirtualFunction" << std::endl;}
    void SetI(const int i) {i_ = i;}
    int GetI() const {return i_;}
};

// A class derived from MyClass
class MyDerivedClass : public MyClass {
public:
    // Implemented virtual function
    virtual void VirtualFunction() const override {std::cout << "Derived::VirtualFunction" << std::endl;}
    // Override a function that is not virtual
    void NotVirtualFunction() const {std::cout << "Derived::NotVirtualFunction" << std::endl;}
};

// Another class derived from MyClass
class MyOtherDerivedClass : public MyClass {
public:
    // Implemented virtual function
    virtual void VirtualFunction() const override {std::cout << "OtherDerived::VirtualFunction" << std::endl;}
    // Override a function that is not virtual
    void NotVirtualFunction() const {std::cout << "OtherDerived::NotVirtualFunction" << std::endl;}
};

Here is a function that calls the virtual method on a reference of the base class type:

In [ ]:
// Calls 'VirtualFunction' on the passed in reference
void CallVirtualFunction(MyClass & cls) {
    // Static type of cls is always "MyClass"
    cls.VirtualFunction();
}

And here is a function that calls the non-virtual method on a reference of the base class type:

In [ ]:
// Calls 'NotVirtualFunction' on the passed in reference
void CallNotVirtualFunction(MyClass & cls) {
    cls.NotVirtualFunction();
}

Finally, because MyClass is abstract we cannot create a function that takes a by-value argument of the base class type.

In [ ]:
// Calls 'VirtualFunction' on the passed in class
void CallVirtualFunctionStatic(MyClass cls) {
    // cls is not a reference - it has no dynamic type
    cls.VirtualFunction();
}

Instantiate one of each class:

In [ ]:
MyDerivedClass derived_class;
MyOtherDerivedClass other_derived_class;

This piece of code calls our functions with the instances.  Note that the dynamic type of the references will be the respective classes - whether the overridden function gets called will depend on if the function is virtual (and therefore resolved at runtime) or not (resolved at compile time).

In [ ]:
// Dynamic type of cls in CallVirtualFunction will be MyDerivedClass
CallVirtualFunction(derived_class);
CallNotVirtualFunction(derived_class);

// Dynamic type of cls in CallVirtualFunction will be MyOtherDerivedClass
CallVirtualFunction(other_derived_class);
CallNotVirtualFunction(other_derived_class);

MyClass is astract, so we cannot create instances but if we could then our references could have both dynamic and static types of MyClass.

In [ ]:
MyClass my_class; // Error - MyClass is abstract and cannot be created
// If it was not abstract, then the dynamic type of cls in CallVirtualFunction would be MyClass (same as its static type)
CallVirtualFunction(my_class);
CallNotVirtualFunction(my_class);

### The override keyword

The override keyword protects our programs from silly mistakes, which is a good thing for making sure our programs are correct:

In [ ]:
// A class to illustrate mistakes in overriding
class MyOverrideMistake : public MyClass {
public:
    // I made a mistake, the signature does not match
    virtual void VirtualFunction(const int a) const override;
    // I made a mistake, the function name has a typo
    virtual void VitrualFunction() const override;
};

### The diamond problem

We already have two classes, MyDerivedClass and MyOtherDerivedClass, that derive from a common base, MyClass.  Making an example of the diamond problem is trivial just by deriving from both of them:

In [ ]:
// A class to illustrate the diamond problem
class MyDiamondProblemClass : public MyDerivedClass, public MyOtherDerivedClass {};

Creating an instance, so we can use it to illustrate the problem:

In [ ]:
MyDiamondProblemClass diamond_problem;

Trying to call any of the functions will result in a compile time error:

In [ ]:
diamond_problem.VirtualFunction();

But we can resolve the ambiguity and call just one of the ancestors:

In [ ]:
diamond_problem.MyDerivedClass::VirtualFunction();

In [ ]:
diamond_problem.MyOtherDerivedClass::VirtualFunction();

However our MyDiamondProblemClass class now has two seperate MyClass sub-objects, each with it's own internal state.  This is going to get confusing:

In [ ]:
diamond_problem.MyDerivedClass::SetI(20);
diamond_problem.MyOtherDerivedClass::SetI(17);

In [ ]:
std::cout <<
    "Derived's parent: " << diamond_problem.MyDerivedClass::GetI() <<
    " OtherDerived's parent:" << diamond_problem.MyOtherDerivedClass::GetI() <<
    std::endl;

### Virtual inheritance

Prevents the diamond problem by ensuring only 1 subobject is created for each virtual type:

In [ ]:
// A class derived from MyClass
class MyVirtualDerivedClass : virtual public MyClass {
public:
    // Implemented virtual function
    virtual void VirtualFunction() const override {std::cout << "Derived::VirtualFunction" << std::endl;}
    // Override a function that is not virtual
    void NotVirtualFunction() const {std::cout << "Derived::NotVirtualFunction" << std::endl;}
};

// Another class derived from MyClass
class MyVirtualOtherDerivedClass : virtual public MyClass {
public:
    // Override a function that is not virtual
    void NotVirtualFunction() const {std::cout << "OtherDerived::NotVirtualFunction" << std::endl;}
};

In [ ]:
// A class to illustrate the diamond problem
class MyVirtualDiamondProblemClass : public MyVirtualDerivedClass, public MyVirtualOtherDerivedClass {};

Now create an instance so we can see the problem is resolved:

In [ ]:
MyVirtualDiamondProblemClass diamond_solution;

In [ ]:
diamond_solution.VirtualFunction();

In [ ]:
diamond_solution.SetI(20);

In [ ]:
std::cout <<
    "VirtualDerived's parent: " << diamond_solution.MyVirtualDerivedClass::GetI() <<
    " VirtualOtherDerived's parent:" << diamond_solution.MyVirtualOtherDerivedClass::GetI() <<
    std::endl;

In [ ]:
diamond_solution.MyDerivedClass::SetI(17);

In [ ]:
std::cout <<
    "VirtualDerived's parent: " << diamond_solution.MyVirtualDerivedClass::GetI() <<
    " VirtualOtherDerived's parent:" << diamond_solution.MyVirtualOtherDerivedClass::GetI() <<
    std::endl;

Virtual inheritance only ensures one copy if the class is virually inherited, there will still be seperate sub-objects for each non-virtually inherited version:

In [ ]:
class MyStillADiamondClass :
    public MyVirtualDerivedClass,
    public MyVirtualOtherDerivedClass,
    public MyDerivedClass,
    public MyOtherDerivedClass
{};

In [ ]:
MyStillADiamondClass still_a_diamond_problem;

In [ ]:
still_a_diamond_problem.VirtualFunction(); // Still ambiguous

In [ ]:
still_a_diamond_problem.MyVirtualDerivedClass::SetI(10);

In [ ]:
still_a_diamond_problem.MyDerivedClass::SetI(11);

In [ ]:
still_a_diamond_problem.MyOtherDerivedClass::SetI(12);

In [ ]:
std::cout <<
    "Derived's parent: " << still_a_diamond_problem.MyDerivedClass::GetI() <<
    " OtherDerived's parent:" << still_a_diamond_problem.MyOtherDerivedClass::GetI() <<
    " VirtualDerived's parent: " << still_a_diamond_problem.MyVirtualDerivedClass::GetI() <<
    " VirtualOtherDerived's parent:" << still_a_diamond_problem.MyVirtualOtherDerivedClass::GetI() <<
    std::endl;

In [ ]:
still_a_diamond_problem.MyVirtualOtherDerivedClass::SetI(13);

In [ ]:
std::cout <<
    "Derived's parent: " << still_a_diamond_problem.MyDerivedClass::GetI() <<
    " OtherDerived's parent:" << still_a_diamond_problem.MyOtherDerivedClass::GetI() <<
    " VirtualDerived's parent: " << still_a_diamond_problem.MyVirtualDerivedClass::GetI() <<
    " VirtualOtherDerived's parent:" << still_a_diamond_problem.MyVirtualOtherDerivedClass::GetI() <<
    std::endl;

## Casting

Casting allows us to explicitly change they type:

In [ ]:
MyClass & my_class_ref {dynamic_cast<MyClass&>(diamond_solution)};

In [ ]:
my_class_ref.VirtualFunction();

Dynamic casts, which happen at runtime, make sure the cast is sensible:

In [ ]:
MyVirtualDerivedClass & my_virtual_ref {dynamic_cast<MyVirtualDerivedClass&>(derived_class)};

Static casts, which happen at compile time, will check the objects are related (or convertable):

In [ ]:
MyVirtualDerivedClass my_class_static_cast {static_cast<MyClass>(derived_class)};

...but do not prevent all silly casts:

In [ ]:
MyDiamondProblemClass & my_diamond_cast {static_cast<MyDiamondProblemClass&>(derived_class)};

Remember the compiler warns (or errors, depending on settings) if you assign (for example) a double to an int?:

In [ ]:
int a {2.2};

A static_cast is the solution if you deliberately want to do that (but make sure there is not a more sensible way first!):

In [ ]:
int a {static_cast<int>(2.2)};

reinterpret_cast is outright dangerous:

In [ ]:
int b {5};
long long & ref_b {reinterpret_cast<long long&>(b)};

ref_b is now a reference to b, but is being treated as referring to a long long (at least 64 bits of memory) when it actually refers to something that is only an int (16 or 32 bits, usually). 

In [ ]:
std::cout <<
    "int size: " << sizeof(int) <<
    " long long size: " << sizeof(long long) << std::endl <<
    "b size: " << sizeof(b) <<
    " ref_b size: " << sizeof(ref_b) <<
    std::endl;

This is almost certainly going to result in bad things happening if the program uses ref_b!

const_cast is equally problematic because they mean your program effecively lies about what it does:

In [ ]:
// Says the reference argument is const but actually increments it!
void modify_argument (const int & a) {
    ++const_cast<int&>(a);
}

In [ ]:
std::cout << "a: " << a << " b: " << b << std::endl;

In [ ]:
modify_argument(a);
modify_argument(b);

In [ ]:
std::cout << "a: " << a << " b: " << b << std::endl;

## Putting polymorphism to use

Suppose we want to be able to do numerical integration of functions using a common code to discretise the function but possibly different methods for calculating the quadrature (area bounded by the function and discretise bound).

In [ ]:
#include <functional>

First we need an interface and couple of implementations (using trapezium rule and Simpson's rule as examples) for our calculation methods:

In [ ]:
// Base class for a quadrature method
class Quadrature {
public:
    // Estimate the numerical integral of function f between bounds a and b
    virtual double calculate(const double a, const double b, const std::function<double(double)> & f) const = 0;
};

// Trapezium rule calculator
class QuadTrapezium : virtual public Quadrature {
public:
    virtual double calculate(const double a, const double b, const std::function<double(double)> & f) const override;
};

// Simpson's rule calculator
class QuadSimpsons : virtual public Quadrature {
public:
    virtual double calculate(const double a, const double b, const std::function<double(double)> & f) const override;
};

Now the implementation of the Trapezium rule:

In [ ]:
double QuadTrapezium::calculate (const double a, const double b, const std::function<double(double)> & f) const {
    return 0.5 * (b-a) * (f(a) + f(b));
}

...and Simpson's rule:

In [ ]:
double QuadSimpsons::calculate (const double a, const double b, const std::function<double(double)> & f) const {
    // b-a/6 * (f(a) + 4*f(mid-point) + f(b))
    return (b-a) * (f(a) + 4.0*f(0.5*(a+b)) + f(b))/6;
}

Finally, a function that will integrate a function between 0 and 1 using either method (through polymorphism):

In [ ]:
// Integrate a function (f) between 0 and 1, using numerical methods (method) and the given number of intervals (intervals)
double integrate_0_1(const std::function<double(double)> & f, const Quadrature & method, const unsigned int intervals) {
    double h {1.0/intervals}; // size of each interval
    double result {0}; // result value
    for (int i {0}; i < intervals; ++i)
        result += method.calculate(i*h, (i+1)*h, f);
    return result;
}

We need an instance of each method:

In [ ]:
QuadTrapezium trap;
QuadSimpsons simp;

Test both methods with 5 interals for f(x) = $x^2$:

In [ ]:
std::cout <<
    "Trapezium: " << integrate_0_1([](const double x) -> double {return x*x;}, trap, 5) <<
    " Simpsons: " << integrate_0_1([](const double x) -> double {return x*x;}, simp, 5) <<
    std::endl;